# What this jupyternotebook for?

- ....

# Prepare ENV/Load libraries

## Mount Google Drive

In [ ]:
import os
from google.colab import drive

drive.mount('/content/gdrive')
google_drive_path = "/content/gdrive/MyDrive/"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# change directory to the project path

project_path = os.path.join(google_drive_path, "Colab Notebooks/SER/")
os.chdir(project_path)
os.listdir("./")

['Notebook Template Train.ipynb',
 'utils',
 'fonts',
 'meta_data',
 'dataset',
 '.git',
 '.env',
 'libs',
 'README.md',
 'models',
 '.gitignore',
 'logs',
 'SER Model idea.drawio',
 'deprecated',
 'Untitled0.ipynb',
 'Preprocess Json to CSV.ipynb',
 'Data distribution.ipynb',
 'Notebook Template.ipynb']

## Install libraries

In [ ]:
# !rm -rf tmp && git clone https://github.com/chuan-khuna/my-python-utils.git tmp && cp -R tmp/utils ./ && cp -R tmp/fonts ./ && rm -rf tmp

In [ ]:
!python3 --version

Python 3.8.16


Install latest tensorflow version

In [ ]:
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2
!pip uninstall -y -q tensorflow keras tensorflow-estimator tensorflow-text
!pip install -q tensorflow_datasets
!pip install -q -U tensorflow-text tensorflow

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libcudnn8 is already the newest version (8.1.0.77-1+cuda11.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf2onnx 1.13.0 requires flatbuffers<3.0,>=1.12, but you have flatbuffers 22.12.6 which is incompatible.
onnx 1.13.0 requires protobuf<4,>=3.20.2, but you have protobuf 3.19.6 which is incompatible.


In [ ]:
!pip install matplotlib seaborn -Uq
!pip install pythainlp -q
!pip install keras-tuner -q
!pip install pyYAML -q
!pip install tensorflow-io[tensorflow] tf2onnx onnxruntime -Uq

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-text 2.11.0 requires tensorflow<2.12,>=2.11.0; platform_machine != "arm64" or platform_system != "Darwin", but you have tensorflow 2.9.0 which is incompatible.


In [ ]:
%load_ext tensorboard

# Import Libraries

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib
from matplotlib import patheffects
import seaborn as sns

import json
import yaml
import re
from tqdm.notebook import trange, tqdm

import datetime

import tf2onnx
import onnx
import onnxruntime as rt

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import keras_tuner
import tensorflow_io as tfio

seed_ = 20200218
tf.random.set_seed(seed_)
np.random.seed(seed_)

from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
from utils.vis_utils import *

font_dir = [f"{google_drive_path}/code_assets/fonts/"]
mpl_import_fonts(font_dir)

## Ensure that matplotlib can use subplot mosaic

In [ ]:
def check_version(version_str, major, minor):
    print(version_str)
    version = [int(i) for i in version_str.split('.')]
    assert version[0] >= major and version[1] >= minor

check_version(matplotlib.__version__, 3, 6)
check_version(sns.__version__, 0, 12)

del check_version

matplotlib.__version__, sns.__version__, tf.__version__

3.6.2
0.12.2


('3.6.2', '0.12.2', '2.9.0')

## View hardware spec

In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
tf.config.list_physical_devices('GPU')

for device in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(device, True)

In [ ]:
def get_timestamp():
    return datetime.datetime.strftime(datetime.datetime.now(), "%Y%m%d-%H%M")

get_timestamp()

'20221231-1000'

# Load Dataset

Copy dataset from Google Drive to VM storage

In [ ]:
# !rm -rf /content/dataset/

In [ ]:
def unzip_dataset(zipfile, target_loc):
    """Unzip a zip file from google drive to VM storage

    I hypothesise that Google Drive has some limits to read files.
    So I decided to extract file to VM storage instead

    ```python
    target_loc = dataset_path = "/content/dataset/"
    ```

    Args:
        zipfile (str): _description_
        target_loc (str): target root folder to place content in the zip file.
    """

    # create root path to store the dataset
    if not os.path.exists(target_loc):
        os.makedirs(target_loc, exist_ok=True)

    subfolder = zipfile.split('/')[-1].split('.zip')[0]
    target_path = os.path.join(target_loc, subfolder)
    print(f"Unzip {zipfile} to {target_path}")
    os.system(f"unzip {zipfile} -d {target_path}")

path_to_zipfiles = "./dataset/"
dataset_path = "/content/dataset/"

if not(os.path.exists(dataset_path) and len(os.listdir(dataset_path)) == 10):
    for filename in os.listdir(path_to_zipfiles):
        if filename.endswith(".zip"):
            zip_path = os.path.join(path_to_zipfiles, filename)
            unzip_dataset(zip_path, dataset_path)

print(os.listdir(dataset_path))

['studio11-20', 'studio41-50', 'studio1-10', 'zoom1-10', 'zoom11-20', 'studio51-60', 'studio71-80', 'studio61-70', 'studio31-40', 'studio21-30']


In [ ]:
df = pd.read_csv("./meta_data/dataset.csv")

# drop emotion label that tagged as None
df = df[df['majority_emo'] != 'None']
df = df[df['majority_emo'] != 'other']

# drop corrupted files
df = df[df['duration'] > 0]

df = df[df['agreement'] >= 0.5]

# shuffle
df = df.sample(frac=1)

In [ ]:
assert (df['duration'] > 0).all()
df['majority_emo'].drop_duplicates()

10460    Frustrated
5293          Angry
27635       Neutral
24610         Happy
3849            Sad
Name: majority_emo, dtype: object

In [ ]:
df

,path,file,assigned_emo,majority_emo,agreement,duration
10460,/content/dataset/zoom1-10/zoom007/mic/z007_mic...,z007_mic_actor049_impro4_4.flac,Angry,Frustrated,1.000000,13.984
26154,/content/dataset/studio21-30/studio022/con/s02...,s022_con_actor079_impro8_13.flac,Frustrated,Frustrated,0.600000,4.776
5293,/content/dataset/studio41-50/studio050/con/s05...,s050_con_actor140_impro10_6.flac,Angry,Angry,1.000000,4.776
3128,/content/dataset/studio41-50/studio049/con/s04...,s049_con_actor138_script3_1_2a.flac,Angry,Frustrated,0.600000,5.283
27635,/content/dataset/studio21-30/studio025/con/s02...,s025_con_actor090_impro6_16.flac,Happy,Neutral,1.000000,0.826
...,...,...,...,...,...,...
5424,/content/dataset/studio41-50/studio050/con/s05...,s050_con_actor139_impro8_13.flac,Frustrated,Frustrated,0.916667,9.052
19068,/content/dataset/studio71-80/studio076/con/s07...,s076_con_actor191_impro14_1.flac,Angry,Angry,0.812500,7.346
1432,/content/dataset/studio11-20/studio016/con/s01...,s016_con_actor032_impro1_23.flac,Angry,Angry,1.000000,5.410
10822,/content/dataset/zoom1-10/zoom006/mic/z006_mic...,z006_mic_actor047_impro1_18.flac,Neutral,Neutral,0.800000,1.306
